# Welcome to the Simple Fit NEMS tutorial
This short notebook will go over some initial topics, functions, and classes that are important to getting started using NEMS. If you'd like more general information, please check out our github page: https://github.com/LBHB/NEMS

NEMS is based around a lot of the work done through TensorFlow, so it may be worth looking into TensorFlow itself if you need help with the fundamentals behind this: https://www.tensorflow.org/

**Note:** Make sure to compile this notebook using you nems-env

**This is a work in progress**

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from nems import Model
from nems.layers import STRF, DoubleExponential, StateGain
from nems.models import LN_STRF

#This is specifically for notebooks only
%matplotlib inline

## Section 1: Setting up your data
As with any other Neural Network, your data will need to be recorded, imported, and formatted in a way that is actually usable by NEMS itself. We do provide example fake data below, and have a database you can access by taking a look at the example on our github page. 

In [2]:
# Fast-running toy fit options for demonstrations.
options = {'options': {'maxiter': 2, 'ftol': 1e-2}}

- All processed data will need to be in the form of a 2D numpy array
- Layers typically use the structure (T,N) where T is an axis of Time and N is an axis of some form of channel, or section
- Layers may also use a structure (T, S) where T is Time, and S is an axis that represents some state

There are of course other layers with their own structures, and your own layers with varying sets of inputs/outputs. But this will cover most inital and basic layers to get things started.

In [3]:
########################################################
#
# Typical nems.Layers structure:
# layer_example(TIME, CHANNEL)
#
# (X Axis, 2D Numpy Array): TIME can be any representation relevent to your data
# (Y Axis, 2D Numpy Array): CHANNEL is some seperation of data inputs ie... Neuron, Sepctral Channel, etc...
#
# Examples: 
#   1. Spiking responses of neurons is set up as shape(TIME, NEURONS)
#   2. Pupil Size is represented as shape(TIME, PUPIL_STATES)
# See more at: https://temp.website.net/nems.Layers
########################################################

###########################
# A dummy representation of potential LBHB data
#   Spectrogram: A representation of sound stimulus and neuron response
#   Pupil_size: A measurement of arousal
###########################
def my_data_loader(file_path):
    print(f'Loading data from {file_path}, but not really...')

    # TIME = Representation of some x time to use in our layers
    # CHANNELS = Representation of some y channels for # of inputs in our layers
    TIME = 1000
    CHANNELS = 18

    # Creation of random 2D numpy array with X time representation and Y channel representations
    spectrogram = np.random.rand(TIME, CHANNELS)
    # Using our Spectrogram to create a target set of data that our model will attempt to fit
    response = np.stack(spectrogram[:, :5])
    # An example of using states to fit our model together
    pupil_size = np.random.rand(TIME, 1)
    
    return spectrogram, response, pupil_size, TIME, CHANNELS

# Create variables from our data import function
spectrogram, response, pupil_size, TIME, CHANNELS = my_data_loader('/path/to/my/data.csv')

Loading data from /path/to/my/data.csv, but not really...


Here is a set of graphs that represent our Spectrogram data, so you can see what our data looks like before anything happens

In [ ]:
raw_plot, ax = plt.subplots(3, 3, figsize=(12,8))
for i in range(0, 9):
    ax[int(np.ceil(i/3)-1)][i%3].plot(range(0,TIME), (spectrogram[:, i]*10).astype(int)) 
plt.show()

## Section 2: Creating your first model

A model in NEMS is a class imported directly from NEMS. It contains a variety of functions and initial parameters to help get your model initialized and set up.

- add_layers will allow you to provide a set of layers for the model to use.
    - These layers may be sequential, or can have specified inputs/outputs as well depending on the type
    - Layers can also be specified during the initalization of your model in the form Model(layers=[list_of_layers])

In [5]:
model = Model()
model.add_layers(
    STRF(shape=(25,18)),    # Full-rank STRF, 25 temporal x 18 spectral channels
    DoubleExponential(shape=(5,)) # Double-exponential nonlinearity, 100 outputs
)

## Section 3: Fitting the model
Once you have a model created and layers set up, you can start to fit your data through the model itself. 

Our fitting function will take a given **input**(In this case, a 2D array of 18 channels at time 1000) and fit it to a given **target**(A second 2D array with 1 channel and time 1000).

- Fitter options can also be provided, like defined at the top of this page. Although you may want to look at: (TEMP) for more info on what could be done
- The function itself will attempt to fit the input to our target, which can then be used for predictions or plots data

In [ ]:
fit_model = model.fit(input=spectrogram, target=response,
                      fitter_options=options)
fig = fit_model.plot(spectrogram,target=response, figure_kwargs={'figsize': (12,8)})

In [ ]:
model.add_layers(StateGain(shape=(1,1)))
state_fit = model.fit(input=spectrogram, target=response, state=pupil_size, backend='scipy',
                      fitter_options=options)

## Section 4: Predicting with our model

Now that a model is fitted, we can look to predict some form of data.

- Predict will take an input, and other parameters depending on the layers, to provide an output prediction of the input data
- This can also vary depending on the layers, number of inputs, and number of outputs

In [ ]:
prediction = state_fit.predict(spectrogram, state=pupil_size)
fig = state_fit.plot(spectrogram, state=pupil_size, target=response, figure_kwargs={'figsize': (12,8)})

## Section 5: Provided models
While we provide all the tools to create your own layers and models, we also provide a pre-built model as well. 
LN_STRF will start with a set of time_bins and channels for it's initialization, but then your free to immediately start fitting and predicting on this model.

In [ ]:
prefit_model = LN_STRF(time_bins=TIME, channels=CHANNELS)
fitted_LN = prefit_model.fit(input=spectrogram, target=response, output_name='pred')
prefit_prediction = prefit_model.predict(spectrogram)
fig = fitted_LN.plot(spectrogram, target=response, figure_kwargs={'figsize': (12,8)})

## Section 6: Viewing your data
We also provide tool to view your data and plot inputs/outputs for your layers.

- Plot will take in sets of inputs for your model, and provided KWargs to create graphs of data that show the provided layers and their outputs throughout the models cycle.
- This allows you to track the changes in your data as it gets fitted to the model and predicted. It also gives you the oppurtunity to build the plot as you see fit with figure_kwargs

**I also provided a sample of channels from our fake data, so you can see what data is being pushed into the model before it has done anything**

In [ ]:
fig = state_fit.plot(spectrogram, state=pupil_size, target=response, figure_kwargs={'figsize': (12,8)})
fig = fitted_LN.plot(spectrogram, target=response, figure_kwargs={'figsize': (12,8)})
fig = fit_model.plot(spectrogram,target=response, figure_kwargs={'figsize': (12,8)})

In [ ]:
raw_plot, ax = plt.subplots(3, 3, figsize=(12,8))
for i in range(0, 9):
    ax[int(np.ceil(i/3)-1)][i%3].plot(range(0,TIME), (spectrogram[:, i]*10).astype(int)) 
plt.show()